In [1]:
import pandas as pd
pd.set_option('display.max_columns', 100)
import numpy as np
import sys
import os
from subprocess import run, PIPE
import shlex

In [2]:
# folder path to attack and normal pcap tcp dump

# foler contain attack tcp dump files
ATTACK_PCAP_DIR='data/attack_pcap'
# folder contain attack csv files
ATTACK_CSV_DIR='data/attack_csv'
# foler contain normal tcp dump files
NORMAL_PCAP_DIR='data/normal_pcap'
# folder contain normal csv files
NORMAL_CSV_DIR='data/normal_csv'

# kdd99 extractor: tcp dumpt to kdd-like connection record
CONVERTER= 'tools/kdd99_feature_extractor/build-files/src/kdd99extractor'
CONVERTER_ARGS= '-e'

KDD_COLUMNS=['duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land',
         'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised',
         'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells',
         'num_access_files', 'num_outbound_cmds', 'is_host_login', 'is_guest_login', 'count',
         'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate',
         'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count',
         'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate',
         'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'label']
COLUMNS=['duration', 'protocol_type', 'service', 'flag', 'src_bytes',
        'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'count', 'srv_count',
        'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate',
        'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
        'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate',
        'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate',
        'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
        'dst_host_srv_rerror_rate', 'src_ip', 'src_port', 'dst_ip', 'dst_port',
        'last_packet_timestamp', 'attack_type', 'attack_method', 'label' ]

ATTACK_SUBSET=COLUMNS[:28]+COLUMNS[-3:]
NORMAL_SUBSET=COLUMNS[:28]+ COLUMNS[-1:]

In [3]:
def pcap2csv(pcap_dir, csv_dir):
    "Convert tcp dump data to kdd99-like connection records as csv file"
    pcap_files=os.listdir(pcap_dir)
    for filename in pcap_files:
        in_file=os.path.join(pcap_dir, filename)
        out_file=os.path.join(csv_dir, filename.replace('.pcap', '.csv'))
        print(f"Converting {in_file} to {out_file}")
        cmd = f"{CONVERTER} {CONVERTER_ARGS} {in_file} > {out_file}"
        os.system(cmd)
        print('done')
        print('-----------------')

In [4]:
# convert attack tcp dump to csv
pcap2csv(ATTACK_PCAP_DIR, ATTACK_CSV_DIR)

Converting data/attack_pcap/dos-tcp_rst_flood-1.pcap to data/attack_csv/dos-tcp_rst_flood-1.csv
done
-----------------
Converting data/attack_pcap/dos-tcp_syn_flood-rand_2.pcap to data/attack_csv/dos-tcp_syn_flood-rand_2.csv
done
-----------------
Converting data/attack_pcap/probe-other-2.pcap to data/attack_csv/probe-other-2.csv
done
-----------------
Converting data/attack_pcap/dos-http_slow-slowloris_1.pcap to data/attack_csv/dos-http_slow-slowloris_1.csv
done
-----------------
Converting data/attack_pcap/dos-tcp_fin_flood-rand_1.pcap to data/attack_csv/dos-tcp_fin_flood-rand_1.csv
done
-----------------
Converting data/attack_pcap/mitm-arp_poisoning-1.pcap to data/attack_csv/mitm-arp_poisoning-1.csv
done
-----------------
Converting data/attack_pcap/dos-tcp_syn_flood-1.pcap to data/attack_csv/dos-tcp_syn_flood-1.csv
done
-----------------
Converting data/attack_pcap/dos-http_slow-slowloris_2.pcap to data/attack_csv/dos-http_slow-slowloris_2.csv
done
-----------------
Converting dat

In [5]:
# convert normal tcp dump to csv
pcap2csv(NORMAL_PCAP_DIR, NORMAL_CSV_DIR)

Converting data/normal_pcap/normal-1.pcap to data/normal_csv/normal-1.csv
done
-----------------
Converting data/normal_pcap/2.pcap to data/normal_csv/2.csv
done
-----------------
Converting data/normal_pcap/5.pcap to data/normal_csv/5.csv
done
-----------------
Converting data/normal_pcap/4.pcap to data/normal_csv/4.csv
done
-----------------
Converting data/normal_pcap/normal-2.pcap to data/normal_csv/normal-2.csv
done
-----------------
Converting data/normal_pcap/7.pcap to data/normal_csv/7.csv
done
-----------------
Converting data/normal_pcap/3.pcap to data/normal_csv/3.csv
done
-----------------
Converting data/normal_pcap/normal-4.pcap to data/normal_csv/normal-4.csv
done
-----------------
Converting data/normal_pcap/normal-3.pcap to data/normal_csv/normal-3.csv
done
-----------------
Converting data/normal_pcap/1.pcap to data/normal_csv/1.csv
done
-----------------
Converting data/normal_pcap/8.pcap to data/normal_csv/8.csv
done
-----------------
Converting data/normal_pcap/6.p

### Process and lable attack connection

In [6]:
attack_df=pd.DataFrame(columns=COLUMNS)
attack_df

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,src_ip,src_port,dst_ip,dst_port,last_packet_timestamp,attack_type,attack_method,label


In [7]:
for file in os.listdir(ATTACK_CSV_DIR):
    print(f'Processing {file}')
    filename=file.split('.')[0]
    attack_type, attack_method, _=filename.split('-')
    try:
        df=pd.read_csv(os.path.join(ATTACK_CSV_DIR, file), names=COLUMNS)
    except EmptyDataError:
        print('{file} contains no data!')
        continue
    print(df.shape)
    df['attack_type']=attack_type
    df['label']=attack_method
    attack_df=pd.concat([attack_df, df], ignore_index=True)
    print(attack_df.shape)
#     print(attack_df.columns)


Processing dos-tcp_syn_flood-land.csv
(4, 36)
(4, 36)
Processing probe-other-1.csv
(2228, 36)
(2232, 36)
Processing dos-http_flood-hulk_1.csv
(24863, 36)
(27095, 36)
Processing probe-udp_scan-1.csv
(995, 36)
(28090, 36)
Processing probe-icmp_ping_scan-1.csv
(1320, 36)
(29410, 36)
Processing dos-tcp_fin_flood-rand_1.csv
(115079, 36)
(144489, 36)
Processing dos-tcp_syn_flood-1.csv
(58354, 36)
(202843, 36)
Processing dos-tcp_rst_flood-1.csv
(256138, 36)
(458981, 36)
Processing probe-tcp_null_scan-1.csv
(9823, 36)
(468804, 36)
Processing dos-udp_flood-1.csv
(226271, 36)
(695075, 36)
Processing probe-other-2.csv
(4380, 36)
(699455, 36)
Processing dos-http_slow-slowloris_2.csv
(35, 36)
(699490, 36)
Processing mitm-mac_flooding-1.csv
(0, 36)
(699490, 36)
Processing mitm-arp_poisoning-2.csv
(0, 36)
(699490, 36)
Processing r2l-guest_passwd-1.csv
(231, 36)
(699721, 36)
Processing dos-icmp_flood-1.csv
(2, 36)
(699723, 36)
Processing mitm-ping_of_death-1.csv
(2, 36)
(699725, 36)
Processing mitm-ar

In [8]:
attack_df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,src_ip,src_port,dst_ip,dst_port,last_packet_timestamp,attack_type,attack_method,label
0,7,tcp,ftp,S1,19715542,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,100.10.0.2,1156,100.10.0.2,21,2020-06-08T03:06:03,dos,NaN,tcp_syn_flood
1,5,tcp,http,S1,12564398,0,1,0,0,1,0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1,0,0.00,1.00,0.0,0.0,1.0,0.0,0.0,0.0,100.10.0.2,80,100.10.0.2,80,2020-06-08T03:06:40,dos,NaN,tcp_syn_flood
2,2,tcp,http,S1,7070294,0,0,0,0,1,1,1.0,1.0,0.0,0.0,1.0,0.0,0.0,2,1,0.50,0.50,0.0,0.0,1.0,1.0,0.0,0.0,100.10.0.2,21,100.10.0.2,80,2020-06-08T03:06:47,dos,NaN,tcp_syn_flood
3,3,tcp,http,S1,9209662,0,0,0,0,1,1,1.0,1.0,0.0,0.0,1.0,0.0,0.0,3,2,0.67,0.33,0.0,0.0,1.0,1.0,0.0,0.0,100.10.0.2,10000,100.10.0.2,80,2020-06-08T03:07:09,dos,NaN,tcp_syn_flood
4,0,tcp,http,OTH,54,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,192.168.2.5,47994,100.10.0.68,80,2020-06-08T16:20:36,probe,NaN,other


In [9]:
attack_df.describe(include='all')

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,src_ip,src_port,dst_ip,dst_port,last_packet_timestamp,attack_type,attack_method,label
count,864324.0,864324,864324,864324,864324.0,864324.0,864324.0,864324.0,864324.0,864324.0,864324.0,864324.000000,864324.000000,864324.0,864324.0,864324.000000,864324.000000,864324.000000,864324.0,864324.0,864324.000000,864324.000000,864324.000000,864324.000000,864324.000000,864324.000000,864324.0,864324.0,864324,864324.0,864324,864324.0,864324,864324,0.0,864324
unique,58.0,3,46,6,1518.0,104.0,2.0,1.0,3.0,140067.0,140065.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101.0,101.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,482204,65536.0,13465,1799.0,1266,4,NaN,14
top,0.0,tcp,ssh,OTH,54.0,0.0,0.0,0.0,0.0,69446.0,69444.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,192.168.2.3,0.0,100.10.0.2,22.0,2020-06-08T10:50:11,dos,NaN,tcp_rst_flood
freq,819080.0,632262,256445,382869,267517.0,629144.0,864323.0,864324.0,864273.0,31577.0,27323.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,768327.0,759893.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,325910,14020.0,835946,256452.0,36007,834969,NaN,256138
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.220798,0.221644,0.0,0.0,0.975714,0.007137,0.025239,NaN,NaN,0.975904,0.006605,0.023516,0.024592,0.226744,0.227496,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.394280,0.395012,0.0,0.0,0.144363,0.065045,0.149379,NaN,NaN,0.144780,0.063302,0.138213,0.147109,0.401814,0.402363,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.0,0.0,1.000000,0.000000,0.000000,NaN,NaN,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.0,0.0,1.000000,0.000000,0.000000,NaN,NaN,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.180000,0.0,0.0,1.000000,0.000000,0.000000,NaN,NaN,1.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
attack_df[attack_df['attack_type']=='dos']['duration'].count()/attack_df['duration'].count()

0.9660370416649312

In [11]:
attack_df['attack_method'].value_counts()/attack_df.shape[0]*100

Series([], Name: attack_method, dtype: float64)

In [12]:
attack_df=attack_df[ATTACK_SUBSET]

In [13]:
attack_df.drop_duplicates(inplace=True, ignore_index=True)

In [14]:
attack_df.describe(include='all')

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack_type,attack_method,label
count,468298.0,468298,468298,468298,468298.0,468298.0,468298.0,468298.0,468298.0,468298.0,468298.0,468298.000000,468298.000000,468298.0,468298.0,468298.000000,468298.000000,468298.000000,468298.0,468298.0,468298.000000,468298.000000,468298.000000,468298.000000,468298.000000,468298.000000,468298.0,468298.0,468298,0.0,468298
unique,58.0,3,46,6,1518.0,104.0,2.0,1.0,3.0,140067.0,140065.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101.0,101.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,14
top,0.0,tcp,ftp,S1,254.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dos,NaN,tcp_syn_flood
freq,423580.0,357087,154189,196328,154095.0,233564.0,468297.0,468298.0,468247.0,5864.0,701.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,412735.0,404042.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,442915,NaN,212454
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.406674,0.408232,0.0,0.0,0.975218,0.012260,0.023270,NaN,NaN,0.975614,0.011497,0.026403,0.022937,0.417641,0.419026,0.0,0.0,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.458997,0.459525,0.0,0.0,0.138504,0.084411,0.140843,NaN,NaN,0.139179,0.083331,0.144565,0.139337,0.466483,0.466786,0.0,0.0,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.0,0.0,1.000000,0.000000,0.000000,NaN,NaN,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.0,0.0,1.000000,0.000000,0.000000,NaN,NaN,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000,0.0,0.0,1.000000,0.000000,0.000000,NaN,NaN,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.0,0.0,NaN,NaN,NaN


In [15]:
attack_df[attack_df['attack_type']=='dos']['duration'].count()/attack_df['duration'].count()

0.9457973341761016

## Process and label normal connection

In [16]:
normal_df=pd.DataFrame(columns=COLUMNS)

In [17]:
for file in os.listdir(NORMAL_CSV_DIR):
    print(f'Processing {file}')
    try:
        df=pd.read_csv(os.path.join(NORMAL_CSV_DIR, file), names=COLUMNS)
    except EmptyDataError:
        print('{file} contains no data!')
        continue
    print(df.shape)
    df['label']='normal.'
    normal_df=pd.concat([normal_df, df], ignore_index=True)
    print(normal_df.shape)

Processing normal-3.csv
(979, 36)
(979, 36)
Processing 7.csv
(1925, 36)
(2904, 36)
Processing 3.csv
(587, 36)
(3491, 36)
Processing 6.csv
(5994, 36)
(9485, 36)
Processing 4.csv
(5378, 36)
(14863, 36)
Processing normal-2.csv
(19, 36)
(14882, 36)
Processing 8.csv
(10052, 36)
(24934, 36)
Processing normal-4.csv
(821, 36)
(25755, 36)
Processing 2.csv
(1736, 36)
(27491, 36)
Processing normal-1.csv
(160, 36)
(27651, 36)
Processing 5.csv
(2417, 36)
(30068, 36)
Processing 1.csv
(633, 36)
(30701, 36)


In [18]:
normal_df.drop_duplicates(inplace=True, ignore_index=True)

In [19]:
normal_df.describe(include='all')

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,src_ip,src_port,dst_ip,dst_port,last_packet_timestamp,attack_type,attack_method,label
count,30700.0,30700,30700,30700,30700.0,30700.0,30700.0,30700.0,30700.0,30700.0,30700.0,30700.000000,30700.000000,30700.0,30700.0,30700.000000,30700.000000,30700.000000,30700.0,30700.0,30700.000000,30700.000000,30700.000000,30700.000000,30700.000000,30700.000000,30700.0,30700.0,30700,30700.0,30700,30700.0,30700,0.0,0.0,30700
unique,630.0,3,12,6,5150.0,5660.0,1.0,1.0,1.0,292.0,290.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,79.0,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,190,13886.0,802,676.0,17951,NaN,NaN,1
top,0.0,udp,other,SF,66.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,37.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,192.168.0.102,443.0,192.168.0.1,443.0,2020-06-09T08:07:18,NaN,NaN,normal.
freq,19577.0,16516,15103,18763,3829.0,13900.0,30700.0,30700.0,30700.0,17488.0,6508.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5187.0,579.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19864,2794.0,6657,12186.0,217,NaN,NaN,30700
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002315,0.004921,0.0,0.0,0.419595,0.010763,0.476192,NaN,NaN,0.796511,0.034531,0.146366,0.555368,0.002464,0.003719,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.047266,0.068393,0.0,0.0,0.491601,0.093682,0.471268,NaN,NaN,0.385949,0.142159,0.272676,0.428223,0.046684,0.051103,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,NaN,NaN,0.920000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.500000,NaN,NaN,1.000000,0.000000,0.000000,0.770000,0.000000,0.000000,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.0,0.0,1.000000,0.000000,1.000000,NaN,NaN,1.000000,0.000000,0.180000,0.970000,0.000000,0.000000,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
normal_df

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,src_ip,src_port,dst_ip,dst_port,last_packet_timestamp,attack_type,attack_method,label
0,0,tcp,other,OTH,66,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0,0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,192.168.0.109,42924,64.233.187.188,5228,2020-06-09T08:02:20,NaN,NaN,normal.
1,0,tcp,other,OTH,68,0,0,0,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,1.00,0,1,0.0,0.0,0.0,1.00,0.0,0.0,0.0,0.0,64.233.187.188,5228,192.168.0.109,42924,2020-06-09T08:02:21,NaN,NaN,normal.
2,0,tcp,http_443,OTH,66,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0,0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,192.168.0.109,41368,216.58.197.110,443,2020-06-09T08:02:23,NaN,NaN,normal.
3,0,tcp,http_443,OTH,54,0,0,0,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,1.00,0,1,0.0,0.0,0.0,1.00,0.0,0.0,0.0,0.0,192.168.0.109,42256,172.67.70.188,443,2020-06-09T08:02:23,NaN,NaN,normal.
4,0,tcp,http_443,OTH,66,0,0,0,0,0,2,0.0,0.0,0.0,0.0,0.0,0.0,1.00,0,2,0.0,0.0,0.0,1.00,0.0,0.0,0.0,0.0,192.168.0.109,35396,123.30.180.60,443,2020-06-09T08:02:23,NaN,NaN,normal.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30695,0,udp,domain_u,SF,90,127,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,29,29,1.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,192.168.0.109,33678,192.168.0.1,53,2020-06-15T22:54:31,NaN,NaN,normal.
30696,29,udp,other,SF,8125,174514,0,0,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,1.00,0,68,0.0,0.0,0.0,1.00,0.0,0.0,0.0,0.0,192.168.0.109,39993,172.217.25.14,443,2020-06-15T22:55:01,NaN,NaN,normal.
30697,0,udp,domain_u,SF,102,118,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.00,30,30,1.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,192.168.0.109,33449,192.168.0.1,53,2020-06-15T22:54:33,NaN,NaN,normal.
30698,29,udp,other,SF,3714,6838,0,0,0,0,2,0.0,0.0,0.0,0.0,0.0,0.0,1.00,0,67,0.0,0.0,0.0,1.00,0.0,0.0,0.0,0.0,192.168.0.109,40339,172.217.174.202,443,2020-06-15T22:55:03,NaN,NaN,normal.


In [21]:
attack_df

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack_type,attack_method,label
0,7,tcp,ftp,S1,19715542,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.0,dos,NaN,tcp_syn_flood
1,5,tcp,http,S1,12564398,0,1,0,0,1,0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1,0,0.00,1.00,0.0,0.0,1.00,0.00,0.0,0.0,dos,NaN,tcp_syn_flood
2,2,tcp,http,S1,7070294,0,0,0,0,1,1,1.0,1.0,0.0,0.0,1.0,0.0,0.0,2,1,0.50,0.50,0.0,0.0,1.00,1.00,0.0,0.0,dos,NaN,tcp_syn_flood
3,3,tcp,http,S1,9209662,0,0,0,0,1,1,1.0,1.0,0.0,0.0,1.0,0.0,0.0,3,2,0.67,0.33,0.0,0.0,1.00,1.00,0.0,0.0,dos,NaN,tcp_syn_flood
4,0,tcp,http,OTH,54,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.0,probe,NaN,other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
468293,3,tcp,http,S1,222,0,0,0,0,8,8,1.0,1.0,0.0,0.0,1.0,0.0,0.0,47,47,1.00,0.00,0.0,0.0,0.55,0.55,0.0,0.0,dos,NaN,http_slow
468294,3,tcp,http,S1,222,0,0,0,0,9,9,1.0,1.0,0.0,0.0,1.0,0.0,0.0,48,48,1.00,0.00,0.0,0.0,0.56,0.56,0.0,0.0,dos,NaN,http_slow
468295,3,tcp,http,S1,222,0,0,0,0,10,10,1.0,1.0,0.0,0.0,1.0,0.0,0.0,49,49,1.00,0.00,0.0,0.0,0.57,0.57,0.0,0.0,dos,NaN,http_slow
468296,3,tcp,http,S1,222,0,0,0,0,11,11,1.0,1.0,0.0,0.0,1.0,0.0,0.0,50,50,1.00,0.00,0.0,0.0,0.58,0.58,0.0,0.0,dos,NaN,http_slow


In [22]:
total_df=pd.concat([normal_df, attack_df], ignore_index=True)

In [23]:
total_df['label'].value_counts()

tcp_syn_flood          212454
udp_flood              105583
tcp_rst_flood           72402
normal.                 30700
tcp_fin_flood           28096
http_flood              24291
other                    6361
tcp_null_scan            6329
tcp_connection_scan      6255
udp_scan                 5024
icmp_ping_scan           1234
guest_passwd              178
http_slow                  87
ping_of_death               2
icmp_flood                  2
Name: label, dtype: int64

In [24]:
total_df=total_df[NORMAL_SUBSET]

In [25]:
total_df.sample(20)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
414919,0,tcp,ftp,S1,254,58,0,0,0,80353,80352,1.0,1.0,0.0,0.0,1.0,0.0,0.00,100,100,1.0,0.0,0.00,0.00,1.0,1.0,0.0,0.0,tcp_syn_flood
58296,0,tcp,http,OTH,54,0,0,0,0,0,149,0.0,0.0,0.0,0.0,0.0,0.0,1.00,0,100,0.0,0.0,0.51,1.00,0.0,0.0,0.0,0.0,icmp_ping_scan
443984,0,tcp,ftp,S1,254,58,0,0,0,109418,109416,1.0,1.0,0.0,0.0,1.0,0.0,0.00,100,100,1.0,0.0,0.00,0.00,1.0,1.0,0.0,0.0,tcp_syn_flood
411682,0,tcp,ftp,S1,254,58,0,0,0,77116,77115,1.0,1.0,0.0,0.0,1.0,0.0,0.00,100,100,1.0,0.0,0.00,0.00,1.0,1.0,0.0,0.0,tcp_syn_flood
246097,0,udp,other,SF,152,0,0,0,0,19642,19642,0.0,0.0,0.0,0.0,1.0,0.0,0.00,100,100,1.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,udp_flood
434552,0,tcp,ftp,S1,254,0,0,0,0,99986,99984,1.0,1.0,0.0,0.0,1.0,0.0,0.00,100,100,1.0,0.0,0.00,0.00,1.0,1.0,0.0,0.0,tcp_syn_flood
30759,0,tcp,http,OTH,54,0,0,0,0,0,55,0.0,0.0,0.0,0.0,0.0,0.0,1.00,0,55,0.0,0.0,0.62,1.00,0.0,0.0,0.0,0.0,other
176205,0,tcp,ssh,OTH,54,0,0,0,0,30532,30532,0.0,0.0,0.0,0.0,1.0,0.0,0.00,100,100,1.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,tcp_rst_flood
210586,0,tcp,ssh,OTH,54,0,0,0,0,64913,64913,0.0,0.0,0.0,0.0,1.0,0.0,0.00,100,100,1.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,tcp_rst_flood
404805,0,tcp,ftp,S1,254,58,0,0,0,70239,70238,1.0,1.0,0.0,0.0,1.0,0.0,0.00,100,100,1.0,0.0,0.00,0.00,1.0,1.0,0.0,0.0,tcp_syn_flood


### Since the number of normal samples is too small as compared to attack samples, use part of kdd99 normal data additionally

In [26]:
kdd=pd.read_csv('data/kddcup.data', names=KDD_COLUMNS)
kdd.drop_duplicates(inplace=True)
kdd.describe(include='all')

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
count,1.074992e+06,1074992,1074992,1074992,1.074992e+06,1.074992e+06,1.074992e+06,1.074992e+06,1.074992e+06,1.074992e+06,1.074992e+06,1.074992e+06,1.074992e+06,1.074992e+06,1.074992e+06,1.074992e+06,1.074992e+06,1.074992e+06,1.074992e+06,1074992.0,1.074992e+06,1.074992e+06,1.074992e+06,1.074992e+06,1.074992e+06,1.074992e+06,1.074992e+06,1.074992e+06,1.074992e+06,1.074992e+06,1.074992e+06,1.074992e+06,1.074992e+06,1.074992e+06,1.074992e+06,1.074992e+06,1.074992e+06,1.074992e+06,1.074992e+06,1.074992e+06,1.074992e+06,1074992
unique,NaN,3,70,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23
top,NaN,tcp,http,SF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,normal.
freq,NaN,978540,580507,786840,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,812814
mean,1.348908e+02,NaN,NaN,NaN,5.837540e+03,4.873730e+03,2.418623e-05,2.735834e-03,3.627934e-05,5.427947e-02,1.460476e-04,6.301163e-01,3.570817e-02,3.023278e-04,1.674431e-04,5.875300e-02,5.416784e-03,3.386072e-04,4.320032e-03,0.0,1.860479e-06,3.802819e-03,4.934544e+01,1.297961e+01,1.901168e-01,1.908070e-01,7.811688e-02,7.793043e-02,7.785663e-01,3.159951e-02,1.179070e-01,1.619723e+02,1.592091e+02,6.700944e-01,4.949892e-02,9.335762e-02,2.173602e-02,1.906580e-01,1.901296e-01,7.954099e-02,7.829662e-02,NaN
std,1.321605e+03,NaN,NaN,NaN,2.009617e+06,1.376866e+06,4.917892e-03,8.835986e-02,1.540162e-02,9.974618e-01,1.558112e-02,4.827732e-01,8.232096e+00,1.738496e-02,1.725250e-02,8.406153e+00,2.650495e-01,1.864927e-02,7.347828e-02,0.0,1.363993e-03,6.154966e-02,8.523218e+01,2.941398e+01,3.909777e-01,3.916605e-01,2.669225e-01,2.659105e-01,3.917200e-01,1.169086e-01,2.655164e-01,1.022645e+02,1.098842e+02,4.259518e-01,1.310801e-01,2.298291e-01,5.636309e-02,3.907621e-01,3.913478e-01,2.621995e-01,2.609840e-01,NaN
min,0.000000e+00,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN
25%,0.000000e+00,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,2.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,5.000000e+01,1.800000e+01,7.000000e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN
50%,0.000000e+00,NaN,NaN,NaN,2.190000e+02,3.320000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,8.000000e+00,7.000000e+00,0.000000e+00,0.000000e+00,0

In [27]:
normal_kdd=kdd[kdd['label']=='normal.'].reset_index(drop=True)
focus=normal_kdd[NORMAL_SUBSET]
focus.sample(20)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
199764,0,tcp,http,SF,236,1610,0,0,0,7,23,0.0,0.0,0.0,0.0,1.0,0.00,0.09,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,normal.
345305,0,tcp,ftp_data,SF,8325,0,0,0,0,14,14,0.0,0.0,0.0,0.0,1.0,0.00,0.00,63,94,0.38,0.10,0.38,0.02,0.00,0.00,0.0,0.00,normal.
505903,0,tcp,http,SF,286,5902,0,0,0,17,17,0.0,0.0,0.0,0.0,1.0,0.00,0.00,37,255,1.00,0.00,0.03,0.04,0.00,0.00,0.0,0.01,normal.
412662,0,tcp,http,SF,218,1122,0,0,0,11,11,0.0,0.0,0.0,0.0,1.0,0.00,0.00,11,255,1.00,0.00,0.09,0.02,0.00,0.00,0.0,0.00,normal.
376151,0,udp,domain_u,SF,46,139,0,0,0,236,312,0.0,0.0,0.0,0.0,1.0,0.01,0.01,255,244,0.96,0.01,0.00,0.00,0.00,0.00,0.0,0.00,normal.
50403,0,tcp,http,SF,232,348,0,0,0,15,27,0.0,0.0,0.0,0.0,1.0,0.00,0.07,30,255,1.00,0.00,0.03,0.06,0.00,0.00,0.0,0.00,normal.
211797,0,tcp,http,SF,247,1408,0,0,0,9,9,0.0,0.0,0.0,0.0,1.0,0.00,0.00,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,normal.
438762,0,tcp,http,SF,287,2491,0,0,0,19,19,0.0,0.0,0.0,0.0,1.0,0.00,0.00,178,255,1.00,0.00,0.01,0.01,0.00,0.00,0.0,0.00,normal.
280806,0,tcp,http,SF,229,1733,0,0,0,24,24,0.0,0.0,0.0,0.0,1.0,0.00,0.00,63,255,1.00,0.00,0.02,0.02,0.00,0.00,0.0,0.00,normal.
59838,0,tcp,http,SF,216,644,0,0,0,11,11,0.0,0.0,0.0,0.0,1.0,0.00,0.00,11,255,1.00,0.00,0.09,0.09,0.00,0.00,0.0,0.00,normal.


In [28]:
focus.sample(5)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
20074,0,tcp,http,SF,232,6850,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.00,20,255,1.00,0.00,0.05,0.04,0.0,0.0,0.0,0.0,normal.
151703,0,tcp,http,SF,206,873,0,0,0,14,17,0.0,0.0,0.0,0.0,1.0,0.0,0.18,59,255,1.00,0.00,0.02,0.01,0.0,0.0,0.0,0.0,normal.
478132,0,tcp,smtp,SF,1046,340,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.00,95,135,0.66,0.04,0.01,0.01,0.0,0.0,0.0,0.0,normal.
203013,0,tcp,http,SF,269,4271,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.00,2,255,1.00,0.00,0.50,0.01,0.0,0.0,0.0,0.0,normal.
139823,0,tcp,http,SF,219,283,0,0,0,5,5,0.0,0.0,0.0,0.0,1.0,0.0,0.00,34,255,1.00,0.00,0.03,0.04,0.0,0.0,0.0,0.0,normal.


In [29]:
total_df=pd.concat([total_df, focus], ignore_index=True)
total_df.drop_duplicates(inplace=True)
total_df.describe(include='all')

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
count,1306823.0,1306823,1306823,1306823,1306823.0,1306823.0,1306823.0,1306823.0,1306823.0,1306823.0,1306823.0,1.306823e+06,1.306823e+06,1.306823e+06,1.306823e+06,1.306823e+06,1.306823e+06,1.306823e+06,1306823.0,1306823.0,1.306823e+06,1.306823e+06,1.306823e+06,1.306823e+06,1.306823e+06,1.306823e+06,1.306823e+06,1.306823e+06,1306823
unique,9070.0,3,48,10,9799.0,23630.0,2.0,1.0,7.0,140067.0,140065.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,256.0,256.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15
top,0.0,tcp,http,SF,254.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,255.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,normal.
freq,1165350.0,1095317,690144,924138,156143.0,337830.0,1306815.0,1306823.0,1306759.0,261351.0,179868.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,414537.0,510588.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,838525
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.468691e-01,1.476688e-01,2.850301e-02,2.869914e-02,9.722149e-01,1.447245e-02,1.125174e-01,NaN,NaN,9.007750e-01,2.796490e-02,7.996012e-02,3.621608e-02,1.512192e-01,1.510151e-01,2.999681e-02,2.909875e-02,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.373157e-01,3.379267e-01,1.655957e-01,1.643657e-01,1.486384e-01,1.015170e-01,2.660401e-01,NaN,NaN,2.512305e-01,1.162931e-01,2.137324e-01,1.358796e-01,3.438952e-01,3.441303e-01,1.622720e-01,1.580536e-01,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,5.000000e-02,NaN,NaN,1.000000e+00,0.000000e+00,3.000000e-02,2.000000e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN


In [30]:
total_df['label'].value_counts()

normal.                838525
tcp_syn_flood          212454
udp_flood              105583
tcp_rst_flood           72402
tcp_fin_flood           28096
http_flood              24291
other                    6361
tcp_null_scan            6329
tcp_connection_scan      6255
udp_scan                 5024
icmp_ping_scan           1234
guest_passwd              178
http_slow                  87
ping_of_death               2
icmp_flood                  2
Name: label, dtype: int64

In [31]:
total_df.to_csv('data/mydata.csv')